In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../../../")
import os
from pathlib import Path

from povertymapping.rollout_grids import get_region_filtered_bingtile_grids, compute_raster_stats
from povertymapping.iso3 import get_region_name

import pandas as pd
import geopandas as gpd

/home/jace/workspace/unicef-ai4d-poverty-mapping/env/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


# Generate Rollout Grids: Myanmar

This notebook generates the rollout grid tiles over the country. The output file is used as an input for Step 3, where we will run the trained model over the set of grids.

The generated grids are set at 2.4km (zoom level 14), matching the grids used during model training. The grids are also filtered to only include populated areas based on a gridded population data.

**Notes on Myanmar**

As of time of writing, HRSL population data for Myanmar is not available in the HDX site, which will cause `hdx.get_hdx_file` to fail. We use population data downloaded from [WorldPop](https://data.humdata.org/dataset/worldpop-population-counts-for-myanmar) instead to filter the grids.

## Set country-specific parameters

In [3]:
COUNTRY_CODE = "mm"
REGION = get_region_name(COUNTRY_CODE, code='alpha-2').lower()
ADMIN_LVL = 'ADM2'
ZOOM_LEVEL = 14
GROUP_ZOOM_LEVEL = 10

# Replace with your Worldpop file location
pop_file = Path('../../../data/worldpop/mm/mmr_ppp_2020_UNadj_constrained.tif')

## Generate Grids

In [4]:
admin_grids_gdf = get_region_filtered_bingtile_grids(
    REGION, 
    admin_lvl=ADMIN_LVL, 
    quadkey_lvl=ZOOM_LEVEL, 
    use_cache=True,
    filter_population=False # We set this to False to not trigger an HDX download
)

2023-04-05 15:39:51.311 | INFO     | povertymapping.rollout_grids:get_region_filtered_bingtile_grids:259 - Loading cached grids file /home/jace/.cache/geowrangler/quadkey_grids/myanmar_14_ADM2_admin_grids.geojson


## Explore Populated Grids

In [5]:
admin_grids_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 133637 entries, 0 to 133636
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   quadkey     133637 non-null  object  
 1   shapeName   133637 non-null  object  
 2   shapeISO    133637 non-null  object  
 3   shapeID     133637 non-null  object  
 4   shapeGroup  133637 non-null  object  
 5   shapeType   133637 non-null  object  
 6   geometry    133637 non-null  geometry
dtypes: geometry(1), object(6)
memory usage: 7.1+ MB


In [6]:
admin_grids_gdf.head(2)

,quadkey,shapeName,shapeISO,shapeID,shapeGroup,shapeType,geometry
0,13220212231203,Yangon (South),None,MMR-ADM2-3_0_0-B72,MMR,ADM2,"POLYGON ((93.36182 14.13658, 93.36182 14.15788..."
1,13220212231221,Yangon (South),None,MMR-ADM2-3_0_0-B72,MMR,ADM2,"POLYGON ((93.36182 14.11527, 93.36182 14.13658..."


In [ ]:
admin_grids_gdf.explore()

## Compute Population Per Grid

For this calculation, we will calculate the population, batched based on groups calculated from the quadkey. By getting the first n digits of the quadkey, we are able to get the zoom level n quadkey to which that tile belongs to. 

Ex. if we group based on zoom level 10 the quadkey `31000101131223` belongs to the grouping `3100010113`.

This grouping ensures that the tile groupings are geographically close to one another, which reduces the raster window size that we needed to calculate the population count. 

For more information about quadkeys, see [Bing Maps Tile System](https://learn.microsoft.com/en-us/bingmaps/articles/bing-maps-tile-system).

In [7]:
quadkey_group_col = f'quadkey_level{GROUP_ZOOM_LEVEL}'
admin_grids_gdf[quadkey_group_col] = admin_grids_gdf['quadkey'].str[:GROUP_ZOOM_LEVEL]
admin_grids_gdf.head(3)

,quadkey,shapeName,shapeISO,shapeID,shapeGroup,shapeType,geometry,quadkey_level10
0,13220212231203,Yangon (South),None,MMR-ADM2-3_0_0-B72,MMR,ADM2,"POLYGON ((93.36182 14.13658, 93.36182 14.15788...",1322021223
1,13220212231221,Yangon (South),None,MMR-ADM2-3_0_0-B72,MMR,ADM2,"POLYGON ((93.36182 14.11527, 93.36182 14.13658...",1322021223
2,13220212231223,Yangon (South),None,MMR-ADM2-3_0_0-B72,MMR,ADM2,"POLYGON ((93.36182 14.09396, 93.36182 14.11527...",1322021223


### Compute population count, batched by the quadkey group

In [8]:
# For Worldpop data only: set aggregation settings
aggregation=dict(
    column='population',
    output='pop_count',
    func='sum')
extra_args=dict(nodata=-99999)

In [9]:
# For Worldpop data only: compute population totals per grid
admin_grids_pop_count = compute_raster_stats(
    admin_grids_gdf,
    pop_file,
    aggregation=aggregation,
    extra_args=extra_args,
    group_col=quadkey_group_col,
    max_batch_size=None,
    n_workers=2
)

2023-04-05 15:40:08.243 | INFO     | povertymapping.rollout_grids:compute_raster_stats:71 - Creating raster zonal stats for 133637 grids for file size 8.619787 Mb, batched in 647 unique group/s from quadkey_level10
2023-04-05 15:40:08.244 | WARNING  | povertymapping.rollout_grids:compute_raster_stats:74 - When batching by group, output gdf rows will be ordered based on the group.
100%|██████████| 647/647 [05:33<00:00,  1.94it/s]
2023-04-05 15:45:41.357 | INFO     | povertymapping.rollout_grids:compute_raster_stats:89 - Completed raster zonal stats for 647 groups
2023-04-05 15:45:41.436 | INFO     | povertymapping.rollout_grids:compute_raster_stats:91 - Concatenated raster zonal stats for 647 groups


### Keep grids with total population > 0

In [10]:
admin_grids_filtered = admin_grids_pop_count[admin_grids_pop_count["pop_count"] > 0]
admin_grids_filtered = admin_grids_filtered.reset_index(drop=True)
admin_grids_filtered.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 27830 entries, 0 to 27829
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   quadkey          27830 non-null  object  
 1   shapeName        27830 non-null  object  
 2   shapeISO         27830 non-null  object  
 3   shapeID          27830 non-null  object  
 4   shapeGroup       27830 non-null  object  
 5   shapeType        27830 non-null  object  
 6   geometry         27830 non-null  geometry
 7   quadkey_level10  27830 non-null  object  
 8   pop_count        27830 non-null  float64 
dtypes: float64(1), geometry(1), object(7)
memory usage: 1.9+ MB


In [11]:
# Uncomment to view interactive map
# admin_grids_filtered.explore()

## Save to file

In [12]:
rollout_date = '-'.join(os.getcwd().split('/')[-2].split('-')[:3])
grid_save_path = f'./{rollout_date}-{COUNTRY_CODE}-rollout-grids.geojson'
admin_grids_filtered.to_file(grid_save_path, driver='GeoJSON', index=False)